In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Read
df = pd.read_csv('Data/cleaned_reviews_actionable.csv')

# Total Reviews
print(f"> Total Reviews: {df.shape[0]:,}")

# View
display(df.head())

> Total Reviews: 100


,reviews_text,sentiment,why
0,the hotel was great. staff went above and beyond bringing hot tea to room at 11:30 pm at no charge!,POSITIVE,Staff provided exceptional service by delivering complimentary hot tea late night.
1,"a wonderful hotel - would definitely stay there whenever we are in alexandria again. wonderful staff. we remarked that the first room was very far from the elevator, and they immediately moved us to a much nicer/larger room across from the elevator without question. i hope i am able to stay at a kimpton hotel on future trips.",POSITIVE,"Excellent staff service, room upgrade, and overall wonderful experience."
2,"i needed a place to stay in va that was close to or in alexandria and the monaco was available for a great value. i enjoyed my stay, room was comfortable and clean. was disappointed on parking cost but i was able to work around that. very convenient to old town.",POSITIVE,"Guest appreciated value, comfort, cleanliness, and location despite parking concerns."
3,"the hotel itself was very nice. the desk staff were very attentive and helpful. the rooms were nice but not dusted/cleaned adequately each day. the problems we had were few but definitely impacted whether we would stay there again. the hallways and elevators were not air-conditioned and were in the high 80's in july. also, with no complimentary breakfast offered, the free coffee was in high demand. the coffee hours ended at 9:00 a.m. and the coffee was always empty, missing ""to go"" cups, or out of cream and sugar. a daily conversation with the desk staff on this issue grew tiresome with our 7 day stay. look elsewhere if service is important to you.",NEGATIVE,"Multiple service failures including poor housekeeping, inadequate climate control, and unreliable complimentary coffee."
4,"we arrive around 1 am, for a 5 night stay, and the lady could not find our reservation. after a while she she found it but could not tell us a thing about our expedia gold benefits. she told us it was not her problem a we should ask the manager eventually.",NEGATIVE,Staff unable to locate reservation and dismissive about guest benefits.


#### AGGREGATE DATA - THE WHY

In [3]:
# Concatenate all 'why' statements into a single string separated by newlines
aggregated_reasons = "\n".join(df['why'].dropna())

# View
aggregated_reasons[:2000]

'Staff provided exceptional service by delivering complimentary hot tea late night.\nExcellent staff service, room upgrade, and overall wonderful experience.\nGuest appreciated value, comfort, cleanliness, and location despite parking concerns.\nMultiple service failures including poor housekeeping, inadequate climate control, and unreliable complimentary coffee.\nStaff unable to locate reservation and dismissive about guest benefits.\nGuest praised location, value, cleanliness, and comfort with enthusiastic tone.\nPositive amenities and location offset by multiple noise and maintenance issues.\nClean, stylish rooms and decor outweigh accessibility concerns.\nGuest praised cleanliness, decor, amenities, location, and expressed intent to return.\nStaff excellence, beautiful rooms, and prime location create exceptional guest experiences.\nStaff was attentive and thoughtful with special touches during busy inauguration event.\nExcellent location, great staff, and overall satisfaction desp

#### MODEL

In [4]:
from langchain.chat_models import init_chat_model

# MODEL
model = init_chat_model(model="claude-haiku-4-5-20251001", temperature=0)

#### PROMPT

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    You are an expert hospitality data analyst.
    
    Analyse the following list of reasons for hotel review sentiments.
    
    Generate a concise summary of actionable operational insights.
    
    Limit the output to a maximum of 5 bullet points.
    
    Reasons: {reasons_text}
    """
)

#### PARSER

In [6]:
from langchain_core.output_parsers import StrOutputParser

# Establish the parsing method for unstructured text output
parser = StrOutputParser()

#### EXECUTION CHAIN

In [7]:
summary_chain = prompt | model | parser

In [8]:
# Execute Chain
actionable_insights = summary_chain.invoke({"reasons_text": aggregated_reasons})

#### INSIGHTS

In [9]:
# VIEW
print(actionable_insights)

# Hospitality Data Analysis: Key Operational Insights

• **Prioritize Housekeeping & Maintenance Standards** – Cleanliness and room condition are consistently praised but operational failures (faulty thermostats, broken amenities, poor housekeeping, low water pressure) significantly damage guest satisfaction. Implement rigorous quality control protocols and preventive maintenance schedules.

• **Strengthen Staff Training & Consistency** – Exceptional staff service is the strongest positive driver across reviews, yet inconsistencies exist (dismissive behavior, impolite cleaning staff, poor manager responses). Invest in comprehensive training programs and accountability measures to ensure consistent excellence across all departments.

• **Address Parking & Hidden Fees Friction** – Unexpected parking charges and undisclosed fees create disproportionate negative sentiment despite otherwise positive stays. Clearly communicate all costs upfront and consider competitive pricing to reduce chec